# Prediction with LSTM model (v.03)
<p style='text-align: right;'>with selectd4.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [11]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

restart kernel... 2025-05-26 16:24:58.203186


In [1]:
# load dependacies
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input

from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.model1 import PredictLSTM1
from lib.model2 import PredictLSTM2
from lib.model3 import PredictLSTM3
from lib.model4 import PredictLSTM4

import argparse
import matplotlib.pyplot as plt

from lib.util_pred import print_data
from lib.util_pred import flat_data_with_sum, flat_data, get_frequency
from lib.util_pred import save_model, import_mode
from lib.util_pred import get_random_in_list
from lib.util_pred import print_data_with_sort
from lib.util_pred import get_sorted_n_values
from lib.util_pred import dict_key_count
from lib.util_pred import print_list
from lib.util_pred import print_dict_list
from lib.util_pred import print_title
from lib.util_pred import listinlist_2_strinlist
from lib.util_pred import change_matched_info
from lib.json_util import write_json

from lib.small_predict import predict_small, print_predict_small

from lib.activation import ActivationOutput, RecurrentActivation
from datetime import datetime

""" db """
from lib.db_operate import conn_db, close_db
from lib.db_operate import init_metric
from lib.db_operate import select_metric, delete_all_metric
from lib.db_operate import insert_metric

from lib.operate import create_model, create_model_v1, get_predicted, create_model_v2

print(f"imported library. ({datetime.now()})")

imported library. (2025-05-26 16:25:10.953649)


In [2]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd4.csv'
    
mode='predict' # help = "back-test or predict")
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=0.95 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 6

write_to_db = True


finished to set environemnt. (2025-05-26 16:25:10.959789)


In [3]:
import tensorflow as tf
print(f'{tf.version}')

<module 'tensorflow._api.v2.version' from '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [4]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }
#print(f'test_X = {dataset_dicts[1].test_X}')
print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-05-26 16:25:14.427138


In [5]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_units=[[(45, ActivationOutput.selu.name),   # 0
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 1
             (45, ActivationOutput.selu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 2
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 3
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 4
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             ],
            [(45, ActivationOutput.tanh.name)    # 5
             ],
            ]
dense_units = [[(45, ActivationOutput.selu.name),   # 0
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.selu.name),], # 1
               [(45, ActivationOutput.selu.name),   # 2
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.tanh.name),   # 3
               ],
               []                                   # 4
              ]
sel_date = datetime(2025, 5, 24, 20, 35)
sel_date_ts = (sel_date.timestamp() / 10000000)
steps = [45, 45,]
metrics = [["accuracy"]]
dropout = [0, 0]
learning_rate = [0.01, 0.05]
last_lstm_return_sequences = [False]
loss = ["binary_crossentropy", "mse"]
output_dense_activation = [ActivationOutput.elu.name, #0
                           ActivationOutput.selu.name, #1
                           ActivationOutput.sigmoid.name, #2
                           ActivationOutput.tanh.name, #3
                           ActivationOutput.softmax.name, #4
                           ActivationOutput.relu.name, #5
                           ActivationOutput.linear.name #6
                          ]
epochs = [10, #0
          20, #1
          25, #2
          50, #3
          100] #4

rand_seed=[sel_date_ts, datetime.now().timestamp(), 0]


lstm_args = {1: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[0],
                          "dense_units": dense_units[0],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             2: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[1],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             3: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[1]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             4: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[2],
                          "dense_units": dense_units[2],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "return_state": False,
                          "rand_seed": rand_seed[0] # rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 90 #100
                           },
                 "dataset": datasets[0]
                 },
             5: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[3],
                          "dense_units": dense_units[3],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             6: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[5],
                          "dense_units": dense_units[4],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "return_state": False,
                          "lstm_model": "Bidirectional",
                          "rand_seed": rand_seed[0] # rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 90 #100
                           },
                 "dataset": datasets[0]
                 }
             }


models = [None for i in range(MAX_MODEL_CNT)]

matched_cnts = {}
selected_fives = {}
matched_list = {}
predicted_all = {}

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-26 16:25:15.431181


In [6]:
single_run = False
model_id=5
if single_run:
    model_version = 0
    with open('active.version', 'r') as fd:
        line = fd.read()
        model_version = int(line)
        print(f'previous model_version={model_version}')
        print(f'open model version file {datetime.now()}')
        
        # single : create model and training
        print(f'start to train all models. {datetime.now()}')
        print(f'{model_id+1}\'s training. status=start {datetime.now()}')
        model = create_model_v2(id=model_id,
                            model_type="lstm4",
                            lstm_info=lstm_args[model_id+1],
                            dataset=lstm_args[model_id+1]["dataset"],
                            verbose=False)
        # models[model_id] = (model, lstm_args[model_id+1])
        print(f'{model_id+1}\'s training. status=end {datetime.now()}')
        print(f'completed to train models. {datetime.now()}')


In [42]:
# all : create model and training
model_version = 0
with open('active.version', 'r') as fd:
    line = fd.read()
    model_version = int(line)
print(f'previous model_version={model_version}')
print(f'open model version file {datetime.now()}')

print(f'start to train all models. {datetime.now()}')
for i in range(MAX_MODEL_CNT):
    print(f'{i+1}\'s training. status=start {datetime.now()}')
    model = create_model_v2(id=i,
                         model_type="lstm4",
                         lstm_info=lstm_args[i+1],
                         dataset=lstm_args[i+1]["dataset"],
                         verbose=False
                        )
    models[i] = (model, lstm_args[i+1])
    print(f'{i+1}\'s training. status=end {datetime.now()}')
print(f'completed to train all models. {datetime.now()}')
model_version += 1
print(f'check to create model({model_version}). {datetime.now()}')
current = 0
data_cols = []
with open('active.version', 'w') as fd:
    fd.write(f'{model_version}')

previous model_version=15
open model version file 2025-05-26 18:25:27.016863
start to train all models. 2025-05-26 18:25:27.016956
1's training. status=start 2025-05-26 18:25:27.017183
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
lstm unit : 3, (45, 'sigmoid')
lstm unit : 4, (45, 'elu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_72, built=True>
num_epoch=50/<class 'int'>
1's training. status=end 2025-05-26 18:26:26.339522
2's training. status=start 2025-05-26 18:26:26.339619
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_73, built=True>
num_epoch=20/<class 'int'>
2's training. status=end 2025-05-26 18:26:51.676847
3's training. status=start 2025-05-26 18:26:51.676946
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_74, built=True>
num_epoch=50/<class 'int'>
3's training. status=end 2025-05-26

In [43]:
print(f'check to create model({model_version}). {datetime.now()}')

check to create model(16). 2025-05-26 18:29:12.048018


In [ ]:
view_model_info = False
if view_model_info:
    for i in range(MAX_MODEL_CNT):
        print(models[i][0].model.summary())

In [ ]:
# all with trial : print predicted
run = False
if run:
    write_json(trial = 3, matched_count=matched_cnts, matched_list=matched_list, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list[i])

In [44]:
write_to_db = True
if write_to_db:
    conn = conn_db('./db/metrics.db')
    need_db_init = False
    init_metric(conn, drop_table=need_db_init)
else:
    conn = None
round_limit=40
for round in range(round_limit+1):
    matched_cnts_all = []
    matched_list = []
    date = str(datetime.now())
    matched_cnts_small, matched_list_small = predict_small(round=round,
                                                           trial=trial,
                                                           last=last,
                                                           models=models,
                                                           max_model_cnt=MAX_MODEL_CNT,
                                                           verbose=False,
                                                           mode2=mode2
                                                          )
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)
    insert_metric(conn=conn,
                  version=model_version,
                  round=round,
                  date=date,
                  model_datas=matched_list)
if write_to_db:
    close_db(conn)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
predicted time : round=0 date=2025-05-26 18:29:22.837971
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
predicted time : round=1 date=2025-05-26 18:29:23.612819
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
predicted time : round=2 date=2025-05-26 18:29:24.302366
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [ ]:
trial = 1
matched_cnts_all = []
matched_list = []
for t in range(1, trial+1):
    matched_cnts_small, matched_list_small = predict_small(trial=t, 
                                                           last=last,
                                                           models=models)
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)
current +=1


In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()

In [ ]:
# all : model info
#################
# model info
#################
for model in models:
    if model is not None:
        print(f'model:\n\t{model[0].model.summary()}')
        print(f'layer:\n\t{model[1]}')

In [ ]:
# all : model test
mode = "back-test"
if mode == 'back-test':
    for model in models:
        if model is not None:
            greed_prediction_number_set = model[0].predict_numbers("greed", trial=1)
            random_pred_set = model[0].predict_randomely(trial=1)
            model[0].evaluate(greed_prediction_number_set)
            print("---------Random baseline-------------")
            model[0].evaluate(random_pred_set)

In [ ]:
for i in range(3):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
# one : create model and training
id = 6
print(f'start to train. [id={id}]')
models[6] = create_model_v1(id=6, dataset=dataset, epoch=10, verbose=True)
print(f'model = {models[6]}')
print(f'end to train. [id={id}]')

In [ ]:
# [3, 6, 7, 11, 12, 17]

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set2, 5)}')

In [ ]:
print_data(title='greed_predicted',
           data_set=greed_prediction_number_set,
           add_val=0, need_sort=True)
print(f'greed_prediction_number_set\n\t\t{greed_prediction_number_set}')

In [ ]:
# when 
"""
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=100 # default = 3
batch=3 # default = 1
model='lstm2' # help = "lstm or lstm2")
hid_dim = 128
"""
for i in range(6):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
print(f'model={model}')

In [ ]:
random_pred_set = model2.predict_randomely(trial=1)
print(f'prediction_number_set = {random_pred_set}')

In [ ]:
# 1170 : 3·13·28·34·38·42

last = [
    [6, 8, 37, 40, 41, 44],
    [8, 10, 20, 25, 33, 37],
    [4, 18, 27, 32, 40, 43],
    [2, 8, 10, 31, 33, 35],
    [15, 20, 28, 38, 41, 45],
    [12, 26, 13, 5, 31, 38]
]

In [ ]:
#1171 : [3, 6, 7, 11, 12, 17]

```
sqlite> select * from sqlite_master;
table|metrics|metrics|2|CREATE TABLE metrics (
            id INTEGER PRIMARY KEY,
            version INTEGER,
            round INTEGER,
            model_id INTEGER, 
            trial INTEGER,
            date DATETIME,
            metric TEXT,
            matched TEXT,
            matched_size INTEGER
        )
```